In [35]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [36]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [8]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [30]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4])


In [31]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0103,  0.0031,  0.0092,  ...,  0.0199, -0.0047,  0.0049],
        [ 0.0041, -0.0151,  0.0054,  ...,  0.0200,  0.0090,  0.0227]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0091, -0.0335], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0370,  0.0360,  0.0366,  ...,  0.0325,  0.0250, -0.0369],
        [ 0.0401, -0.0357,  0.0386,  ..., -0.0121,  0.0072, -0.0343]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 

Autograd is automatic gradient computation. Use `torch.no_grad()` to disable gradient tracking, when you want to freeze params or speed up a forward-only computation.
In the back pass, autograd accumulates gradients in the `.grad` attribute of a tensor. We need to zero the gradients before a new back pass.

## Optimization Loop

In [38]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [39]:
learning_rate = 1e-3
batch_size = 64
epochs = 10

# Initialize the loss function
# common choices: nn.MSELoss(), nn.CrossEntropyLoss(), nn.NLLLoss(), nn.BCELoss()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.296013  [   64/60000]
loss: 2.287519  [ 6464/60000]
loss: 2.270291  [12864/60000]
loss: 2.266093  [19264/60000]
loss: 2.245693  [25664/60000]
loss: 2.211928  [32064/60000]
loss: 2.231156  [38464/60000]
loss: 2.195112  [44864/60000]
loss: 2.178909  [51264/60000]
loss: 2.165900  [57664/60000]
Test Error: 
 Accuracy: 42.6%, Avg loss: 2.153262 

Epoch 2
-------------------------------
loss: 2.159062  [   64/60000]
loss: 2.154088  [ 6464/60000]
loss: 2.100945  [12864/60000]
loss: 2.116356  [19264/60000]
loss: 2.066423  [25664/60000]
loss: 2.004109  [32064/60000]
loss: 2.039397  [38464/60000]
loss: 1.963092  [44864/60000]
loss: 1.943723  [51264/60000]
loss: 1.904632  [57664/60000]
Test Error: 
 Accuracy: 58.8%, Avg loss: 1.890577 

Epoch 3
-------------------------------
loss: 1.912726  [   64/60000]
loss: 1.893266  [ 6464/60000]
loss: 1.779049  [12864/60000]
loss: 1.820216  [19264/60000]
loss: 1.711976  [25664/60000]
loss: 1.661497  [32064/600